# Problema de los ascensores

#### G05 - Boris Carballa Corredoira, Juan Carlos Villanueva Quirós, Francisco Javier Blázquez Martínez

Paralelización del problema, el edificio consta de un único bloque con varios ascensores.

In [237]:
from search import *
import copy

In [238]:
# CONSTANTES
NUM_PLANTAS    = 5 #No se usa
NUM_ASCENSORES = 2
NUM_PERSONAS   = 5 #Se usa en quita_dejar_coger_inutiles_acciones
CAP_ASCENSOR   = 2
EN_ASCENSOR    = -1
NINGUNA        = -1
EN_DESTINO     = -2

In [239]:
%run SolutionManager.ipynb

In [240]:
# El estado de un nodo del espacio de exploración se compone de:
# ascensor_pos      -> Lista con plantas en las que se encuentran los ascensores
# ascensor_personas -> Lista de listas, con las personas en cada ascensor
# personas          -> Lista con planta en la que se encuentra cada persona 
#                      (Pueden ser EN_ASCENSOR o EN_DESTINO)
class NodeState:

    def __init__(self, init_personas):
        self.personas = init_personas
        self.personas_tiempo_llegada = [0 for _ in range(NUM_PERSONAS)]
        self.ascensor_pos = [0 for _ in range(NUM_ASCENSORES)]       # Los ascensores empiezan en la planta cero
        self.ascensores_tiempo_llegada = [0 for _ in range(NUM_ASCENSORES)]
        self.ascensor_personas = [[] for _ in range(NUM_ASCENSORES)] # Los ascensores empiezan vacíos
        
    def __eq__(self, nodo):
        return self.personas==nodo.personas and self.ascensor_pos==nodo.ascensor_pos and self.ascensor_personas==nodo.ascensor_personas and self.personas_tiempo_llegada==nodo.personas_tiempo_llegada and self.ascensores_tiempo_llegada==nodo.ascensores_tiempo_llegada
    
    # Función necesaria para los algoritmos con control de repetidos
    def __hash__(self):
        return hash((tuple(self.personas), tuple(self.personas_tiempo_llegada), tuple(self.ascensor_pos), tuple(self.ascensores_tiempo_llegada),tuple(tuple(x) for x in self.ascensor_personas)))
    
    # Función necesaria para algoritmos como astar_search()
    def __lt__(self,nodo):
        return True

In [241]:
# Definimos la clase para la representación del problema. 

class AscensoresSimplificado(Problem) :    
   
    def __init__(self, initial, goal=None):        
        self.initial = initial    # NodeState
        self.goal = goal          # Lista enteros
        self.analizados = 0

    # Codificación acciones:
    # (BAJAR, i, j) -> El ascensor i baja j plantas
    # (SUBIR, i, j) -> El ascensor i sube j plantas
    # (COGER_PERSONA, i, j) -> El ascensor i coge a la persona j
    # (DEJAR_PERSONA, i, j) -> El ascensor i deja a la persona j
    # (ESPERAR, i, j) -> El ascensor i espera j segundos
    def actions(self, state):
        
        accs = list()
        personas = state.personas
        personas_tiempo_llegada = state.personas_tiempo_llegada
        
        for num_ascensor in range(0,NUM_ASCENSORES):
            # Parámetros relativos al ascensor a evaluar
            ascensor_pos      = state.ascensor_pos[num_ascensor]      # Planta del ascensor
            ascensor_personas = state.ascensor_personas[num_ascensor] # Personas dentro del ascensor
            ascensor_vacio    = not ascensor_personas
            ascensor_lleno    = (len(ascensor_personas)==CAP_ASCENSOR)
            ascensor_tiempo_llegada = state.ascensores_tiempo_llegada[num_ascensor]
            
            # Personas en la misma planta en la que se encuentra el ascensor que suben y bajan
            personas_esta_planta_suben = [i for i in range(len(personas)) if personas[i]==ascensor_pos and self.goal[i]>ascensor_pos]
            personas_esta_planta_bajan = [i for i in range(len(personas)) if personas[i]==ascensor_pos and self.goal[i]<ascensor_pos]


            # 1.- Si alguien ha llegado a su destino, forzamos que baje del ascensor
            for persona in ascensor_personas:
                if ascensor_pos==self.goal[persona]:
                    return [(DEJAR_PERSONA,num_ascensor,persona)]


            # 2.- Para la acción subir requerimos que haya que coger o dejar a alguien
            #     más arriba de donde se encuentra el ascensor. Nunca subimos con gente
            #     que desea bajar plantas y además, un ascensor no sube si tiene capacidad
            #     para más personas y en esa planta hay alguien que desea subir (eliminación
            #     de ramas sin perder optimaldad).

            persona_menos_arriba = min([person for person in personas if person>ascensor_pos and person!=EN_ASCENSOR and person!=EN_DESTINO], default=NINGUNA)

            # Si el ascensor está vacío sólo subimos si hay alguna persona por arriba 
            # y no hay gente en esta planta que quiera subir
            if ascensor_vacio:
                if persona_menos_arriba!=NINGUNA and not(personas_esta_planta_suben):
                    accs.append((SUBIR,num_ascensor,persona_menos_arriba-ascensor_pos))

            # Si el ascensor no está vacío comprobamos que sus personas suben y vamos
            # al mínimo entre a donde suben estas y donde hay una persona por encima
            # siempre que no nos quede espacio o no haya gente en esa planta que quiere subir
            elif self.goal[ascensor_personas[0]]>ascensor_pos:
                destino_menos_arriba = min([self.goal[i] for i in ascensor_personas])

                if ascensor_lleno or not(personas_esta_planta_suben):
                    if persona_menos_arriba==NINGUNA:
                        accs.append((SUBIR,num_ascensor,destino_menos_arriba-ascensor_pos))
                    else:
                        accs.append((SUBIR,num_ascensor,min(persona_menos_arriba, destino_menos_arriba)-ascensor_pos))


            # 3.- La acción bajar es totalmente dual a la acción subir, misma lógica en
            #     las condiciones. 

            persona_menos_abajo = max([person for person in personas if person<ascensor_pos and person!=EN_ASCENSOR and person!=EN_DESTINO], default=NINGUNA)

            # Si el ascensor está vacío sólo bajamos si hay alguna persona por abajo
            if ascensor_vacio:
                if persona_menos_abajo!=NINGUNA and not(personas_esta_planta_bajan):
                    accs.append((BAJAR,num_ascensor,ascensor_pos-persona_menos_abajo))
            elif self.goal[ascensor_personas[0]]<ascensor_pos:
                destino_menos_abajo = max([self.goal[i] for i in ascensor_personas])

                if ascensor_lleno or not(personas_esta_planta_bajan):
                    if persona_menos_abajo==NINGUNA:
                        accs.append((BAJAR,num_ascensor,ascensor_pos-destino_menos_abajo))
                    else:
                        accs.append((BAJAR,num_ascensor,ascensor_pos-max(persona_menos_abajo, destino_menos_abajo)))

            # 4.- Acción dejar persona:
            for personaInterior in ascensor_personas:
                accs.append((DEJAR_PERSONA,num_ascensor,personaInterior))

            # 5.- Acción coger persona. Sólo cogemos a personas que suben o bajan, no ambas
            if not ascensor_lleno:
                if ascensor_vacio:
                    for persona in personas_esta_planta_suben:
                        if personas_tiempo_llegada[persona]>ascensor_tiempo_llegada:
                            accs.append((ESPERAR,num_ascensor,personas_tiempo_llegada[persona]-ascensor_tiempo_llegada))
                        else:
                            accs.append((COGER_PERSONA,num_ascensor,persona))
                    for persona in personas_esta_planta_bajan:
                        if personas_tiempo_llegada[persona]>ascensor_tiempo_llegada:
                            accs.append((ESPERAR,num_ascensor,personas_tiempo_llegada[persona]-ascensor_tiempo_llegada))
                        else:
                            accs.append((COGER_PERSONA,num_ascensor,persona))
                else:
                    if self.goal[ascensor_personas[0]]<ascensor_pos:
                        for persona in personas_esta_planta_bajan:
                            if personas_tiempo_llegada[persona]>ascensor_tiempo_llegada:
                                accs.append((ESPERAR,num_ascensor,personas_tiempo_llegada[persona]-ascensor_tiempo_llegada))
                            else:
                                accs.append((COGER_PERSONA,num_ascensor,persona))
                    else:
                        for persona in personas_esta_planta_suben:
                            if personas_tiempo_llegada[persona]>ascensor_tiempo_llegada:
                                accs.append((ESPERAR,num_ascensor,personas_tiempo_llegada[persona]-ascensor_tiempo_llegada))
                            else:
                                accs.append((COGER_PERSONA,num_ascensor,persona))

        return accs

    def result(self, state, action):
        estado_nuevo = copy.deepcopy(state)
        ascensor = action[1]
        
        if action[0]==BAJAR:
            estado_nuevo.ascensor_pos[ascensor] -= action[2]
            estado_nuevo.ascensores_tiempo_llegada[ascensor] += action[2]
        elif action[0]==SUBIR:
            estado_nuevo.ascensor_pos[ascensor] += action[2]
            estado_nuevo.ascensores_tiempo_llegada[ascensor] += action[2]
        elif action[0]==COGER_PERSONA:
            estado_nuevo.ascensor_personas[ascensor].append(action[2])
            estado_nuevo.personas[action[2]] = EN_ASCENSOR
        elif action[0]==ESPERAR:
            estado_nuevo.ascensores_tiempo_llegada[ascensor] += action[2]
        else:
            estado_nuevo.ascensor_personas[ascensor].remove(action[2])
            estado_nuevo.personas_tiempo_llegada[action[2]]=state.ascensores_tiempo_llegada[ascensor]
            
            if self.goal[action[2]]==state.ascensor_pos[ascensor]:
                estado_nuevo.personas[action[2]] = EN_DESTINO
            else:
                estado_nuevo.personas[action[2]] = state.ascensor_pos[ascensor]
        
        return estado_nuevo
    
    def goal_test(self, state):
        self.analizados +=1
        return state.personas == [EN_DESTINO for persona in state.personas]

    def path_cost(self, c, state1, action, state2):
        return max(state2.ascensores_tiempo_llegada) - max(state1.ascensores_tiempo_llegada)
    
    def h(self,node):
        suma = sum([abs(self.goal[i]-node.state.personas[i]) for i in range(0,len(node.state.personas)) if node.state.personas[i]!=EN_DESTINO and node.state.personas[i]!=EN_ASCENSOR])
        for ascensor in range(0, NUM_ASCENSORES):
            suma += sum([abs(self.goal[persona]-node.state.ascensor_pos[ascensor]) for persona in node.state.ascensor_personas[ascensor]])
        return suma/CAP_ASCENSOR*NUM_ASCENSORES
        
    def value(self, state):
        raise NotImplementedError

In [252]:
#init = NodeState([2,4,1,8,1])
#goal = [3,11,12,1,9]
#problem = AscensoresSimplificado(init, goal)

init = NodeState([0,0,0,5])
goal = [10,10,10,0]
problem = AscensoresSimplificado(init, goal)

In [253]:
#acciones = cocina_acciones(breadth_first_graph_search(problem).solution())
#decodificador_acciones(acciones)

In [254]:
#problem.analizados

In [255]:
#coste_acciones(acciones)

In [256]:
problem.analizados=0
acciones = astar_search(problem).solution()
decodificador_acciones(acciones)

El ascensor 1 coge a la persona 2
El ascensor 1 coge a la persona 1
El ascensor 1 sube 5 plantas
El ascensor 1 sube 5 plantas
El ascensor 1 deja a la persona 2
El ascensor 1 deja a la persona 1
El ascensor 0 coge a la persona 0
El ascensor 0 sube 5 plantas
El ascensor 0 sube 5 plantas
El ascensor 0 deja a la persona 0
El ascensor 1 baja 5 plantas
El ascensor 1 coge a la persona 3
El ascensor 1 deja a la persona 3
El ascensor 1 coge a la persona 3
El ascensor 1 baja 5 plantas
El ascensor 1 deja a la persona 3


In [257]:
problem.analizados

17

In [258]:
coste_acciones(acciones)

30

In [259]:
last_node = problem.initial

for acc in acciones:
    last_node = problem.result(last_node, acc)
    

In [260]:
max(last_node.ascensores_tiempo_llegada)

20

In [261]:
last_node.ascensores_tiempo_llegada

[10, 20]